In [12]:
!pip install pandas
!pip install google.colab

You should consider upgrading via the 'C:\Users\evalv\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [13]:
import pandas as pd

In [14]:
test_comments_post1 = ['1comment1', '1comment2']
test_comments_post2 = ['2comment1', '2comment2']
comments_post1_df = pd.DataFrame({'comment': test_comments_post1})
comments_post2_df = pd.DataFrame({'comment': test_comments_post2})
path_main = 'data'
comments_post1_df.to_csv(path_main + '\\comments1.csv')
comments_post2_df.to_csv(path_main + '\\comments2.csv')


comments_post1_df

,comment
0,1comment1
1,1comment2


In [15]:
comments_post2_df

,comment
0,2comment1
1,2comment2


In [ ]:
from google.colab import files
files.download('data/comments1.csv')
files.download('data/comments2.csv')